# Lucas-Kanade Analysis

In [1]:
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(evaluation_dir, 'lucas_kanade.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])
predictions.head()

,subject,setting,method,use_cgof,grey,duration,sampling_rate,signal
0,Proband01,101_natural_lighting,lucas_kanade_cgof,True,False,0 days 00:00:03.961036,30,"[566.1892881912896, 566.1583861238988, 566.090..."
1,Proband01,101_natural_lighting,lucas_kanade,False,False,0 days 00:00:03.850009,30,"[569.9464284117253, 569.8949780022082, 569.838..."
2,Proband01,101_natural_lighting,lucas_kanade_cgof,True,True,0 days 00:00:00.805029,30,"[569.4346933795572, 569.3766135619596, 569.319..."
3,Proband01,101_natural_lighting,lucas_kanade,False,True,0 days 00:00:00.603969,30,"[576.2930666175787, 576.2187359947692, 576.173..."
4,Proband02,101_natural_lighting,lucas_kanade_cgof,True,False,0 days 00:00:03.834660,30,"[575.59385902618, 575.574281238526, 575.574272..."


## Compare the predictions to the ground truth

In [2]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [3]:
import respiration.analysis as analysis

methods = predictions['method'].unique()
use_cgof = [True, False]
grey = [True, False]

analysis_results = []

for method in methods:
    for g in grey:
        signals = predictions[
            (predictions['method'] == method) &
            (predictions['grey'] == g)]

        experiment_analysis = analysis.Analysis()

        for idx, row in signals.iterrows():
            subject, setting = row['subject'], row['setting']
            prediction = row['signal']
            gt_signal = dataset.get_breathing_signal(subject, setting)
            experiment_analysis.add_data(prediction, gt_signal, 30)

        metrics = experiment_analysis.compute_metrics()
        for entry in metrics:
            analysis_results.append({
                'model': method,
                'grey': g,
                'metric': entry['metric'],
                'method': entry['method'],
                'value': entry['value'],
            })

analysis_results = pd.DataFrame(analysis_results)

In [4]:
# Only shoe MAE and pk
analysis_results[
    (analysis_results['metric'] == 'Correlation')
    # (analysis_results['method'] == 'pk')
    ]

,model,grey,metric,method,value
4,lucas_kanade_cgof,True,Correlation,cp,0.745475
9,lucas_kanade_cgof,True,Correlation,nfcp,0.598218
14,lucas_kanade_cgof,True,Correlation,pk,0.748590
19,lucas_kanade_cgof,True,Correlation,psd,0.761112
24,lucas_kanade_cgof,False,Correlation,cp,0.742582
29,lucas_kanade_cgof,False,Correlation,nfcp,0.616787
34,lucas_kanade_cgof,False,Correlation,pk,0.744566
39,lucas_kanade_cgof,False,Correlation,psd,0.778987
44,lucas_kanade,True,Correlation,cp,0.754250
49,lucas_kanade,True,Correlation,nfcp,0.607507
